In [2]:
import numpy as np
import torch
from torch import Tensor, sort
from sklearn.metrics import label_ranking_loss

In [3]:
#  My implementation
def num_swapped_pairs(ys_true: Tensor, ys_pred: Tensor) -> int:
    
    n_objects = ys_true.shape[0]
    num_swapped_pairs = 0
    
    for i in range(n_objects - 1):
        
        curr_object_true = ys_true[i]
        curr_object_pred = ys_pred[i]
        
        mask_true_objects = (ys_true > curr_object_true)[i+1:].to(torch.int64)
        mask_pred_objects = (ys_pred < curr_object_pred)[i+1:].to(torch.int64)
        
        num_swapped_pairs += (mask_true_objects @ mask_pred_objects)
        
        mask_true_objects = (ys_true < curr_object_true)[i+1:].to(torch.int64)
        mask_pred_objects = (ys_pred > curr_object_pred)[i+1:].to(torch.int64)
        
        num_swapped_pairs += (mask_true_objects @ mask_pred_objects)
        
    return num_swapped_pairs.item()

In [4]:
# Implementation from KC

def num_swapped_pairs(ys_true: Tensor, ys_pred: Tensor) -> int:
    
    ys_pred_sorted, argsort = sort(ys_pred, descending=True, dim = 0)
    ys_true_sorted = ys_true[argsort]
    
    num_objects = ys_true_sorted.shape[0]
    swapped_cnt = 0
    for curr_obj in range(num_objects-1):
        for next_obj in range(curr_obj+1, num_objects):
            if ys_true_sorted[curr_obj] < ys_true_sorted[next_obj]:
                if ys_pred_sorted[curr_obj] > ys_pred_sorted[next_obj]:
                    swapped_cnt += 1
            #elif ys_true_sorted[cur_obj] > ys_true_sorted[next_obj]:
            #    if ys_pred_sorted[curr_obj] < ys_pred_sorted[next_obj]:
            #        swapped_cnt += 1
    
    return swapped_cnt

In [5]:
# Testing  num_swapped_pairs

ys_true = torch.tensor([2, 0, 1])
ys_pred = torch.tensor([0, 2, 1])

num_swapped_pairs(ys_true, ys_pred)

3

In [6]:
def compute_gain(y_value: float, gain_scheme: str) -> float:
    if gain_scheme == 'exp2':
        gain = 2**y_value -  1
    elif gain_scheme == 'const':
        gain = y_value
    else:
        raise ValueError(f'{gain_scheme} method not supported, only exp2 and const')
    return float(gain)

In [7]:
# My implementation

def dcg(ys_true: Tensor, ys_pred: Tensor, gain_scheme: str) -> float:
    
    ys_pred_sorted, argsort = sort(ys_pred, descending=True, dim=0)
    ys_true_sorted = ys_true[argsort]
    
    gain = compute_gain(ys_true_sorted, gain_scheme)
    num_objects = ys_true.shape[0]
    penalty_for_pos = torch.log2(Tensor(range(num_objects)) + 1)
    return (gain / penalty_for_pos).sum()

In [8]:
# Implementation from KC
from math import log2

def dcg(ys_true: Tensor, ys_pred: Tensor, gain_scheme: str) -> float:
    _, argsort = sort(ys_pred, descending=True,dim=0)
    ys_true_sorted = y_true[argsort]
    ret = 0
    for idx, cur_y in enumerate(ys_true_sorted, 1):
        gain = compute_fain(cur_y.item(), gain_scheme)
        ret += gain / log2(idx + 1)
    return ret

In [9]:
# My implmentation ndcg
def ndcg(ys_true: Tensor,  ys_pred: Tensor, gain_scheme: str = 'const') -> float:
    dcg = dcg(ys_true, ys_pred, gain_scheme)
    ideal_dcg =  dcg(ys_true, ys_true, gain_scheme)
    return dcg / ideal_dcg

In [11]:
# Implementation from KC

def precision_at_k(ys_true: Tensor, ys_pred: Tensor, k: int) -> float:
    
    if ys_true.sum() == 0:
        return -1
    _, argsort = torch.sort(ys_pred, descending=True, dim=0)
    ys_true_sorted = ys_true[argsort]
    hits = ys_true_sorted[:k].sum()
    prec = hits / min(ys_true.sum(), k)
    return float(prec)

In [12]:
# Implementation from KC

def reciprocal_rank(ys_true: torch.Tensor, ys_pred: torch.Tensor) -> float:
    _, argsort = torch.sort(ys_pred, descending=True, dim=0)
    ys_true_sorted = ys_true[argsort]
    
    for idx, cur_y in enumerate(ys_true_sorted, 1):
        if cur_y == 1:
            return 1 / idx
    return 0

In [16]:
torch.arange(3) + 1

tensor([1, 2, 3])

In [19]:
# My implementation:

def reciprocal_rank(ys_true: torch.Tensor, ys_pred: torch.Tensor) -> float:
    
    if ys_true.sum() == 0:
        return 0
    
    _, argsort = torch.sort(ys_pred, descending=True, dim=0)
    ys_true_sorted = ys_true[argsort]
    pos = torch.arange(3) + 1
    return 1 / (ys_true_sorted @ pos)

In [20]:
# Implementation from KC

def p_found(ys_true: Tensor, ys_pred: Tensor, p_break: float = 0.15) -> float:
    
    '''
    Note: ys_true is probabilities of relevent for documents
    '''
    
    p_look = 1
    p_found = 0
    _, argsort = torch.sort(ys_pred, descending=True, dim=0)
    ys_true_sorted = ys_true[argsort]
    
    for cur_y in ys_true_sorted:
        pfound += p_look * float(cur_y)
        p_look = p_look * (1 - float(cur_y)) * (1 - p_break)
        
    return p_found

In [21]:
# implementation from KC

def average_precision(ys_true : torch.Tensor, ys_pred: torch.Tensor) -> float:
    if ys_true.sum() == 0:
        return -1
    _, argsort = torch.sort(ys_pred, descending=True,dim=0)
    ys_true_sorted = ys_true[argsort]
    rolling_sum = 0
    num_correct_ans = 0
    for idx, cur_y in enumerate(ys_true_sorted, start=1):
        if cur_y == 1:
            num_correct_ans += 1
            rolling_sum += num_correct_ans / idx
    if num_correct_ans == 0:
        return 0
    else:
        return rolling_sum / num_correct_ans